In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from catboost import CatBoostRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,mean_squared_error
import scipy.stats as stats

In [4]:
df = pd.read_csv('clean_data.csv')

In [5]:
df.head()

,a,b,c,d,target
0,0.005,18.391129,0.0160,0.002839,5.636734
1,0.022,18.391129,0.0182,0.002000,9.098374
2,0.004,18.391129,0.0161,0.003411,4.720418
3,0.007,18.391129,0.0171,0.002606,6.562306
4,0.008,18.391129,0.0161,0.002153,7.478623


In [6]:
X = df.drop(columns=['target'])
y = df['target']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
catboost_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=6)
catboost_model.fit(X_train, y_train)

0:	learn: 2.4272407	total: 51.8ms	remaining: 5.13s
1:	learn: 2.2743266	total: 55.8ms	remaining: 2.74s
2:	learn: 2.1215110	total: 59ms	remaining: 1.91s
3:	learn: 1.9828450	total: 61.1ms	remaining: 1.47s
4:	learn: 1.8748872	total: 64.7ms	remaining: 1.23s
5:	learn: 1.7501949	total: 68ms	remaining: 1.06s
6:	learn: 1.6441114	total: 72.8ms	remaining: 967ms
7:	learn: 1.5465085	total: 75.5ms	remaining: 868ms
8:	learn: 1.4545214	total: 77.7ms	remaining: 786ms
9:	learn: 1.3785533	total: 81.2ms	remaining: 730ms
10:	learn: 1.3025571	total: 84.3ms	remaining: 682ms
11:	learn: 1.2251538	total: 87.4ms	remaining: 641ms
12:	learn: 1.1696569	total: 90.6ms	remaining: 606ms
13:	learn: 1.1186599	total: 93ms	remaining: 571ms
14:	learn: 1.0644222	total: 97ms	remaining: 549ms
15:	learn: 1.0144819	total: 99.7ms	remaining: 523ms
16:	learn: 0.9799303	total: 103ms	remaining: 502ms
17:	learn: 0.9321061	total: 107ms	remaining: 487ms
18:	learn: 0.8919955	total: 110ms	remaining: 468ms
19:	learn: 0.8502395	total: 113ms

In [11]:
y_pred = catboost_model.predict(X_test)

In [12]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.21066321352533043


In [47]:
final_data = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})

final_data.to_csv('Default_parameters_predictions.csv', index=False)

print("Predictions exported to 'Default_parameters_predictions.csv'")

Predictions exported to 'predictions.csv'


In [13]:
y_pred

array([ 7.41363246,  6.22704493,  8.31496944,  3.6538101 ,  5.22883454,
        4.67578067,  4.42944221,  8.56542395,  5.62891678,  2.49427627,
        8.93006548,  7.14240389,  6.99976167,  4.38403638,  4.07724695,
        7.00044278,  5.02270278,  4.99529504,  4.95627613,  3.19181676,
        7.59520613,  9.06821496,  3.56682393,  8.01263599,  3.63480718,
        5.69520446,  2.97596328,  2.85320987,  2.58420478,  5.3355632 ,
        6.51179584,  5.24389464,  5.24920597,  2.5749488 ,  3.55567016,
        8.5404393 ,  2.11041183, 10.09638951,  5.89857448,  7.70079353,
        4.05611219,  5.05737947,  6.42163746,  1.71591607,  7.15892484,
        7.34840877,  2.32448999,  4.22858189,  4.76082706,  4.8273741 ,
        3.65455641,  3.41274769,  3.2009431 ,  2.88043279,  7.52677408,
        3.95415073,  8.22336691,  1.63996518,  4.27226318,  5.0434331 ,
        5.43291066,  4.37646147,  2.53321185,  5.9936137 ,  5.69726873,
        6.85816072,  4.23108866,  2.55006848,  7.69214716, 10.39

In [51]:
# Random Search HPO

from catboost import CatBoostRegressor
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

catboost_params = {
    'iterations': sp_randint(10, 100),
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
    'depth': sp_randint(1, 16),
    'l2_leaf_reg': sp_randint(1, 11),
    'border_count': sp_randint(32, 255)
}

n_iter_search = 2
clf = CatBoostRegressor(random_state=0, verbose=0)
Random = RandomizedSearchCV(clf, param_distributions=catboost_params, n_iter=n_iter_search, cv=3, scoring='neg_mean_squared_error')
Random.fit(X, y)

print("Best Parameters:", Random.best_params_)
print("MSE:", -Random.best_score_)

y_pred = Random.predict(y_test)

import pandas as pd
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).to_csv('Random_Search_HPO_predictions.csv', index=False)

print("Predicted output exported to 'Random_Search_HPO_predictions.csv'")



Best Parameters: {'border_count': 112, 'depth': 10, 'iterations': 61, 'l2_leaf_reg': 5, 'learning_rate': 0.5}
MSE: 0.8465515941746853
Predicted output exported to 'predicted_output.csv'


In [52]:
# Grid Search HPO

from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

catboost_params = {
    'iterations': [10, 20, 30],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [6, 8, 10],
}

clf = CatBoostRegressor(random_state=0, verbose=0)
grid = GridSearchCV(clf, catboost_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)

print("Best Parameters:", grid.best_params_)
print("MSE:", -grid.best_score_)

y_pred = grid.predict(y_test)

import pandas as pd
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).to_csv('Grid_Search_HPO_predictions.csv', index=False)

print("Predicted output exported to 'Grid_Search_HPO_predictions.csv'")


Best Parameters: {'depth': 6, 'iterations': 30, 'learning_rate': 0.1}
MSE: 1.0896539379258723
Predicted output exported to 'Grid_Search_HPO_predictions.csv'


In [27]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.2 MB/s eta 0:00:00


In [54]:
# Bayesian Optimization with Gaussian Process (BO-GP)

from catboost import CatBoostRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.metrics import mean_squared_error

catboost_params = {
    'iterations': Integer(10, 100),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'depth': Integer(1, 16),
    'l2_leaf_reg': Integer(1, 11),
    'border_count': Integer(32, 255)
}

clf = CatBoostRegressor(random_state=0, verbose=0)
Bayes = BayesSearchCV(clf, catboost_params, cv=3, n_iter=2, scoring='neg_mean_squared_error')
Bayes.fit(X, y)

print("Best Parameters:", Bayes.best_params_)
print("MSE:", -Bayes.best_score_)

y_pred = Bayes.predict(y_test)

import pandas as pd
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).to_csv('BO-GP_predictions.csv', index=False)

print("Predicted output exported to 'BO-GP_predictions.csv'")



Best Parameters: OrderedDict([('border_count', 225), ('depth', 9), ('iterations', 73), ('l2_leaf_reg', 9), ('learning_rate', 0.1122346570212007)])
MSE: 1.045993331327972
Predicted output exported to 'BO-GP_predictions.csv'


In [55]:
# Bayesian Optimization with Tree-structured Parzen Estimator (TPE)

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
import numpy as np


def objective(params):
    params = {
        'iterations': int(params['iterations']),
        'learning_rate': params['learning_rate'],
        'depth': int(params['depth']),
        'l2_leaf_reg': int(params['l2_leaf_reg']),
        'border_count': int(params['border_count'])
    }
    clf = CatBoostRegressor(**params, random_state=0, verbose=0)
    scores = -cross_val_score(clf, X, y, cv=3, scoring="neg_mean_squared_error")
    mean_score = np.mean(scores)
    return {'loss': mean_score, 'status': STATUS_OK}


space = {
    'iterations': hp.quniform('iterations', 10, 100, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 1.0),
    'depth': hp.quniform('depth', 1, 16, 1),
    'l2_leaf_reg': hp.quniform('l2_leaf_reg', 1, 11, 1),
    'border_count': hp.quniform('border_count', 32, 255, 1)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=2,
            trials=trials)

print("CatBoost: Hyperopt estimated optimum parameters:", best)

best_params = {
    'iterations': int(best['iterations']),
    'learning_rate': best['learning_rate'],
    'depth': int(best['depth']),
    'l2_leaf_reg': int(best['l2_leaf_reg']),
    'border_count': int(best['border_count'])
}

clf = CatBoostRegressor(**best_params, random_state=0, verbose=0)
clf.fit(X, y)

y_pred = clf.predict(y_test)

import pandas as pd
pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}).to_csv('BO-TPE_predictions.csv', index=False)

print("Predicted output exported to 'BO-TPE_predictions.csv'")


100%|██████████| 2/2 [00:01<00:00,  1.85trial/s, best loss: 0.5039675055435819]
CatBoost: Hyperopt estimated optimum parameters: {'border_count': 222.0, 'depth': 5.0, 'iterations': 66.0, 'l2_leaf_reg': 1.0, 'learning_rate': 0.3751981569908176}
Predicted output exported to 'BO-TPE_predictions.csv'
